In [75]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, cohen_kappa_score, matthews_corrcoef
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold





In [3]:
raw_df = pd.read_csv("../forestCover.csv", na_values="?")
raw_df = raw_df.rename(columns={"Observation_ID":"Water_Level" , "Water_Level":"Observation_ID"})
# display(raw_df)

In [4]:
na_col = raw_df.columns[raw_df.isna().any()]
# print(na_col)
raw_df["Slope"].isna().sum()

np.int64(298)

In [5]:
print(f"Corr of Inclination and Cover: {raw_df['Inclination'].corr(raw_df['Cover_Type'])}")
print(f"Corr of aspect and facet: {raw_df['Facet'].corr(raw_df['Aspect'])}")
print(f"Cardinality of water level: {raw_df['Water_Level'].nunique()}")
print(f"Cardinality of obs id: {raw_df['Observation_ID'].nunique()}")
df = raw_df.drop(columns={'Facet','Water_Level','Observation_ID', 'Inclination' })

na_rows = df.index[df.isna().any(axis=1)]
na_proportions = (df["Cover_Type"][na_rows].value_counts() /  df["Cover_Type"].value_counts() * 100).round(3)
print(na_proportions)
df = df.dropna()

df["Soil_Type1"] = (df["Soil_Type1"] == 'positive').astype(int)
df["Soil_Type1"].value_counts()

Corr of Inclination and Cover: 0.0002417782055169697
Corr of aspect and facet: 0.9999980536139016
Cardinality of water level: 1
Cardinality of obs id: 581012
Cover_Type
1    0.045
2    0.055
3    0.053
4    0.036
5    0.021
6    0.069
7    0.054
Name: count, dtype: float64


Soil_Type1
1    577685
0      3029
Name: count, dtype: int64

In [10]:
continuous_features = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points"
]

# # Plot distributions
# fig, axes = plt.subplots(len(continuous_features), 1, figsize=(5, 15))

# for i, col in enumerate(continuous_features):
#     sns.histplot(df[col], bins=50, kde=True, ax=axes[i], color="steelblue")
#     axes[i].set_title(f"Distribution of {col}")
#     axes[i].set_xlabel("")
#     axes[i].set_ylabel("Count")

# plt.tight_layout()
# plt.show()

In [7]:
df_eng = df.copy()
df_eng["Relative_hoz_dist_mean"] = df[[
    "Horizontal_Distance_To_Hydrology",
    "Horizontal_Distance_To_Fire_Points",
    "Horizontal_Distance_To_Roadways"
]].mean(axis=1)

df_eng["total_hillshade"] = df[["Hillshade_9am","Hillshade_Noon","Hillshade_3pm"]].sum(axis=1)
df_eng["Relative_height_from_water"] = df["Elevation"] - df["Vertical_Distance_To_Hydrology"]
df_eng = df_eng.drop(columns={"Elevation",  "Vertical_Distance_To_Hydrology","Horizontal_Distance_To_Hydrology", "Horizontal_Distance_To_Fire_Points","Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon", "Hillshade_3pm"  })

In [8]:
X = df.drop(columns={"Cover_Type"})
X_eng = df_eng.drop(columns={"Cover_Type"})
y = df["Cover_Type"]
y_eng = df_eng["Cover_Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

eX_train, eX_test, ey_train, ey_test = train_test_split(X_eng, y_eng, test_size=0.3, random_state=42)



In [12]:
outlier_features = [
    "Horizontal_Distance_To_Hydrology",
    "Hillshade_Noon",
    "Vertical_Distance_To_Hydrology",
]

outlier_features_eng = [
    "Relative_hoz_dist_mean",
    
]

knn_df = df.copy()
knn_eng = df_eng.copy()


def clamp_series(s, lower_q=0.01, upper_q=0.99):
    lower, upper = s.quantile([lower_q, upper_q])
    return s.clip(lower, upper)

# apply clamping
for col in outlier_features:
    knn_df[col] = clamp_series(knn_df[col], 0.01, 0.99)
    

knn_eng["Relative_hoz_dist_mean"] = clamp_series(knn_eng["Relative_hoz_dist_mean"], 0.01, 0.99)

    


continuous_features = [

    "Elevation",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
]

continuous_features_eng = [

    'Slope', 
    'Relative_hoz_dist_mean', 
    'total_hillshade',
    'Relative_height_from_water'
]




def scale_knn(knn_df, continuous_features = continuous_features):
    scaler = MinMaxScaler()
    knn_df[continuous_features] = scaler.fit_transform(knn_df[continuous_features])
    knn_df["Aspect_rad"] = np.deg2rad(knn_df["Aspect"])
    knn_df["Aspect_sin"] = np.sin(knn_df["Aspect_rad"])
    knn_df["Aspect_cos"] = np.cos(knn_df["Aspect_rad"])
    knn_df.drop(columns=["Aspect", "Aspect_rad"] , inplace=True)


scale_knn(knn_df)
scale_knn(knn_eng, continuous_features_eng)
    


In [13]:
X = df.drop(columns={"Cover_Type"})
X_eng = df_eng.drop(columns={"Cover_Type"})
y = df["Cover_Type"]
y_eng = df_eng["Cover_Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

eX_train, eX_test, ey_train, ey_test = train_test_split(X_eng, y_eng, test_size=0.3, random_state=42)



In [14]:
kX = df.drop(columns={"Cover_Type"})
kX_eng = df_eng.drop(columns={"Cover_Type"})
ky = df["Cover_Type"]
ky_eng = df_eng["Cover_Type"]

kX_train, kX_test, ky_train, ky_test = train_test_split(X, y, test_size=0.3, random_state=42)

keX_train, keX_test, key_train, key_test = train_test_split(X_eng, y_eng, test_size=0.3, random_state=42)

In [15]:
pca = PCA(n_components=0.9)
X_pca = pca.fit_transform(X)
kX_pca = pca.fit(kX)

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=42)
kX_train_pca, kX_test_pca, ky_train_pca, ky_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=42)

## dealing with class imbalances

In [16]:
tomek = TomekLinks(sampling_strategy='auto')
X_tomek, y_tomek = tomek.fit_resample(X_train, y_train)
eX_tomek, ey_tomek = tomek.fit_resample(eX_train, ey_train)

kX_tomek, ky_tomek = tomek.fit_resample(kX_train, ky_train)
keX_tomek, key_tomek = tomek.fit_resample(keX_train, key_train)


c:\Users\jamie\anaconda3\envs\datsci\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [17]:
smote = SMOTETomek(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)
eX_smote, ey_smote = smote.fit_resample(eX_train, ey_train)
pca_X_smote, pca_y_smote = smote.fit_resample(X_train_pca, y_train_pca)


kX_smote, ky_smote = smote.fit_resample(kX_train, ky_train)
keX_smote, key_smote = smote.fit_resample(keX_train, key_train)
pca_kXsmote, pca_kysmote = smote.fit_resample(kX_train_pca, ky_train_pca)

## evaluation

In [18]:
def evaluate_model(model, X, y, cv=5):
    # Accuracy with mean ± std
    scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
    print(f"Accuracy: {scores.mean():.3f} ± {scores.std():.3f}")
    
    # Cross-validated predictions for confusion matrix etc.
    y_pred = cross_val_predict(model, X, y, cv=cv)
    
    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title(f"Confusion Matrix ({type(model).__name__})")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()
    
    # Classification report (precision, recall, f1 per class)
    print("Classification Report:")
    print(classification_report(y, y_pred, digits=3))
    
    # Cohen’s kappa (agreement beyond chance)
    kappa = cohen_kappa_score(y, y_pred)
    print(f"Cohen’s Kappa: {kappa:.3f}")

def plot_cv_boxplot(model, X, y, cv=5):
    scores = cross_val_score(model, X, y, cv=cv, scoring="accuracy")
    sns.boxplot(data=scores, orient="h", color="skyblue")
    plt.title(f"Accuracy per fold ({type(model).__name__})")
    plt.xlabel("Accuracy")
    plt.show()

from sklearn.model_selection import learning_curve

def plot_learning_curve(model, X, y, cv=5, train_sizes=np.linspace(0.1, 1.0, 5)):
    train_sizes, train_scores, val_scores = learning_curve(
        model, X, y, cv=cv, train_sizes=train_sizes, scoring="accuracy"
    )
    
    train_mean = np.mean(train_scores, axis=1)
    val_mean = np.mean(val_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    val_std = np.std(val_scores, axis=1)
    
    plt.figure(figsize=(7,5))
    plt.plot(train_sizes, train_mean, "o-", label="Training")
    plt.plot(train_sizes, val_mean, "o-", label="Validation")
    plt.fill_between(train_sizes, train_mean-train_std, train_mean+train_std, alpha=0.2)
    plt.fill_between(train_sizes, val_mean-val_std, val_mean+val_std, alpha=0.2)
    plt.title(f"Learning Curve ({type(model).__name__})")
    plt.xlabel("Training set size")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()


## Modeling

In [48]:
def test_knn(X_train , y_train, X_test = kX_test, y_test = ky_test, weights = 'distance' , k = 7, metric = "euclidean"):
    knn = KNeighborsClassifier(n_neighbors=k, metric=metric, weights=weights)
    knn.fit(X_train, y_train)
    train_acc = knn.score(X_train, y_train)
    test_acc = knn.score(X_test, y_test)

    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    

In [49]:
test_knn(kX_tomek, ky_tomek)
test_knn(keX_tomek, key_tomek, X_test=keX_test)

Train Accuracy: 1.0000
Test Accuracy: 0.7988
Train Accuracy: 1.0000
Test Accuracy: 0.7481


In [50]:
display(pca_kysmote)

0         2
1         1
2         1
3         2
4         7
         ..
862031    7
862032    7
862033    7
862034    7
862035    7
Name: Cover_Type, Length: 862036, dtype: int64

In [51]:
test_knn(kX_smote, ky_smote)
test_knn(keX_smote, key_smote, X_test=keX_test)


Train Accuracy: 1.0000
Test Accuracy: 0.7538
Train Accuracy: 1.0000
Test Accuracy: 0.6812


In [52]:
test_knn(pca_kXsmote, pca_kysmote, X_test=X_test_pca)

Train Accuracy: 1.0000
Test Accuracy: 0.3054


In [53]:
test_knn(kX_smote, ky_smote)


Train Accuracy: 1.0000
Test Accuracy: 0.7538


### tree

In [54]:
def test_tree(X_train, y_train, X_test = X_test, y_test = y_test, metric = 'entropy', max_depth = None):
    tree = DecisionTreeClassifier(criterion = metric, max_depth= max_depth, random_state=42)
    tree.fit(X_train, y_train)
    train_acc = tree.score(X_train, y_train)
    test_acc = tree.score(X_test, y_test)

    print(f"Smote train acc: {train_acc}")
    print(f"Smote test acc: {test_acc}")

In [55]:
test_tree(X_tomek, y_tomek)
test_tree(eX_tomek, ey_tomek, X_test=eX_test)

Smote train acc: 1.0
Smote test acc: 0.9339379502339064
Smote train acc: 1.0
Smote test acc: 0.8703039347932153


In [56]:
display(X_test_pca)

array([[-4677.6435595 ],
       [-4592.3962554 ],
       [-4854.59285615],
       ...,
       [-4991.34453709],
       [-4763.26645612],
       [-5025.19968416]], shape=(174215, 1))

In [ ]:
test_tree(X_smote, y_smote)
test_tree(eX_smote, ey_smote, X_test=eX_test)
test_tree(pca_X_smote,pca_y_smote, X_test = X_test_pca)

Smote train acc: 1.0
Smote test acc: 0.9329277042734552
Smote train acc: 1.0
Smote test acc: 0.868082541686996


TypeError: test_tree() got an unexpected keyword argument 'k'

In [ ]:
df_eng = df.copy()
df_eng["ExS"] = df["Elevation"] * df["Slope"]

# Row-wise min/max/mean
df_eng["Relative_hoz_dist_min"] = df[[
    "Horizontal_Distance_To_Hydrology",
    "Horizontal_Distance_To_Fire_Points",
    "Horizontal_Distance_To_Roadways"
]].min(axis=1)

df_eng["Relative_hoz_dist_max"] = df[[
    "Horizontal_Distance_To_Hydrology",
    "Horizontal_Distance_To_Fire_Points",
    "Horizontal_Distance_To_Roadways"
]].max(axis=1)

df_eng["Relative_hoz_dist_mean"] = df[[
    "Horizontal_Distance_To_Hydrology",
    "Horizontal_Distance_To_Fire_Points",
    "Horizontal_Distance_To_Roadways"
]].mean(axis=1)

df_eng["total_hillshade"] = df[["Hillshade_9am","Hillshade_Noon","Hillshade_3pm"]].sum(axis=1)
df_eng["min_hillshade"]   = df[["Hillshade_9am","Hillshade_Noon","Hillshade_3pm"]].min(axis=1)
df_eng["max_hillshade"]   = df[["Hillshade_9am","Hillshade_Noon","Hillshade_3pm"]].max(axis=1)

df_eng["Relative_height_from_water"] = df["Elevation"] - df["Vertical_Distance_To_Hydrology"]


In [58]:
import numpy as np
import pandas as pd
from scipy.stats import skew
from sklearn.feature_selection import f_classif, mutual_info_classif

# ---- 1) Configure your engineered features here ----
eng_cols = [
    "Relative_hoz_dist_mean",
    "total_hillshade",
    "Relative_height_from_water",
]

# df = your original frame, df_eng = engineered columns frame with the cols above
# y should be a 1D array/Series of your class labels (same index as df/df_eng)

def summarize_engineered_features(df_eng: pd.DataFrame, y: pd.Series, eng_cols: list):
    """Return a tidy summary table comparing engineered features:
       descriptives + ANOVA F + Mutual Information vs multiclass target y.
    """
    # Keep numeric only (coerce junk to NaN)
    X = df_eng[eng_cols].apply(pd.to_numeric, errors="coerce")

    # Descriptive statistics
    desc = pd.DataFrame(index=eng_cols)
    desc["count"]     = X.notna().sum()
    desc["missing_%"] = (1 - X.notna().mean()) * 100
    desc["mean"]      = X.mean()
    desc["std"]       = X.std(ddof=1)
    desc["min"]       = X.min()
    desc["q1"]        = X.quantile(0.25)
    desc["median"]    = X.quantile(0.50)
    desc["q3"]        = X.quantile(0.75)
    desc["max"]       = X.max()
    desc["IQR"]       = desc["q3"] - desc["q1"]
    # robust skew (ignore NaN)
    desc["skew"]      = X.apply(lambda s: skew(s.dropna(), bias=False) if s.dropna().size > 2 else np.nan)
    # coefficient of variation (avoid div by zero)
    desc["CV"]        = desc["std"] / desc["mean"].replace({0: np.nan})

    # Predictive signal (align y, drop rows with any NaN in X)
    mask = X.notna().all(axis=1) & pd.notna(y)
    X_clean = X.loc[mask, :]
    y_clean = y.loc[mask]

    # ANOVA F (needs finite)
    F_vals, p_vals = f_classif(X_clean.values, y_clean.values)
    mi_vals = mutual_info_classif(X_clean.values, y_clean.values, discrete_features=False, random_state=42)

    desc["ANOVA_F"] = pd.Series(F_vals, index=eng_cols)
    desc["ANOVA_p"] = pd.Series(p_vals, index=eng_cols)
    desc["MI"]      = pd.Series(mi_vals, index=eng_cols)

    # Order columns nicely
    cols = [
        "mean","std","min","q1","median","q3","max","IQR","skew","CV","missing_%","ANOVA_F","ANOVA_p","MI","count"
    ]
    return desc[cols].sort_values("MI", ascending=False)

def per_class_means(df_eng: pd.DataFrame, y: pd.Series, eng_cols: list):
    """Return per-class means for engineered features (rows=class, cols=features)."""
    tmp = df_eng[eng_cols].copy()
    tmp["__y__"] = y
    return tmp.groupby("__y__")[eng_cols].mean().sort_index()

# --- Helper: pretty formatting for display in the notebook ---
def format_table(df_summary: pd.DataFrame, float_fmt="{:.3f}"):
    fmt = df_summary.copy()
    for c in fmt.columns:
        if pd.api.types.is_float_dtype(fmt[c]):
            fmt[c] = fmt[c].map(lambda v: float_fmt.format(v) if pd.notna(v) else "")
    return fmt

summary = summarize_engineered_features(df_eng, y, eng_cols)
display(format_table(summary))          # main table (sortable in Jupyter if you use DataFrame explorer)

class_means = per_class_means(df_eng, y, eng_cols)
display(format_table(class_means))


,mean,std,min,q1,median,q3,max,IQR,skew,CV,missing_%,ANOVA_F,ANOVA_p,MI,count
Relative_height_from_water,3270051.550,309380.218,2054184.000,3103927.000,3310510.000,3495121.750,4263008.000,391194.750,-0.818,0.095,0.000,155254.905,0.000,0.576,580714
Relative_hoz_dist_mean,3131.804,317524.972,34.000,935.000,1398.000,1971.333,124764839.667,1036.333,257.179,101.387,0.000,1.268,0.268,0.139,580714
total_hillshade,577.995,43.396,95.000,557.000,586.000,609.000,641.000,52.000,-1.290,0.075,0.000,3241.520,0.000,0.019,580714


,Relative_hoz_dist_mean,total_hillshade,Relative_height_from_water
__y__,,,
1,3316.903,579.306,3457108.651
2,3282.190,582.155,3227579.840
3,1336.154,558.115,2645819.212
4,16207.301,556.740,2457373.771
5,4893.251,564.441,3080062.083
6,750.775,550.964,2673175.076
7,1721.608,573.649,3714850.237


In [76]:
depths = [3, 7, 12, 17, 24, 32, 40, None]
best_depth = None
best_test_acc = -np.inf
best_tree = None

for d in depths:  
    tree = DecisionTreeClassifier(criterion = 'entropy', max_depth= d, random_state=42)
    tree.fit(X_train, y_train)
    # train_acc_s = tree.score(X_smote, y_smote)
    train_acc = tree.score(X_train, y_train)
    test_acc = tree.score(X_test, y_test)

    print(f"Depth={d}: train={train_acc:.3f}, test={test_acc:.3f}")
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_depth = d
        best_tree = tree

print("\nBest depth:", best_depth)
print("Best test accuracy:", best_test_acc)

y_pred = best_tree.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))
mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)


Depth=3: train=0.672, test=0.669
Depth=7: train=0.722, test=0.720
Depth=12: train=0.815, test=0.806
Depth=17: train=0.905, test=0.877
Depth=24: train=0.981, test=0.929
Depth=32: train=1.000, test=0.939
Depth=40: train=1.000, test=0.939
Depth=None: train=1.000, test=0.939

Best depth: 40
Best test accuracy: 0.9394828229486554
[[59365  3660     2     0    59    10   268]
 [ 3444 81028   219     1   349   148    43]
 [    6   218  9935    89    32   433     0]
 [    0     0    98   718     0    33     0]
 [   45   369    36     0  2290    12     2]
 [   10   158   417    41    12  4529     0]
 [  289    39     0     0     1     0  5807]]
              precision    recall  f1-score   support

           1      0.940     0.937     0.938     63364
           2      0.948     0.951     0.949     85232
           3      0.928     0.927     0.928     10713
           4      0.846     0.846     0.846       849
           5      0.835     0.832     0.833      2754
           6      0.877     0.87

In [77]:
depths = [3, 7, 12, 17, 24, 32, 40, None]
best_depth = None
best_test_acc = -np.inf
best_tree = None

for d in depths:  
    tree = DecisionTreeClassifier(criterion = 'gini', max_depth= d, random_state=42)
    tree.fit(X_smote, y_smote)
    train_acc_s = tree.score(X_smote, y_smote)
    train_acc = tree.score(X_train, y_train)
    test_acc = tree.score(X_test, y_test)

    print(f"Depth={d}: train={train_acc:.3f}, test={test_acc:.3f}, train_s={train_acc_s:.3f}")
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_depth = d
        s_best_tree = tree

print("\nBest depth:", best_depth)
print("Best test accuracy:", best_test_acc)

y_pred = s_best_tree.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))
mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)


Depth=3: train=0.319, test=0.316, train_s=0.596
Depth=7: train=0.580, test=0.578, train_s=0.732
Depth=12: train=0.747, test=0.737, train_s=0.849
Depth=17: train=0.880, test=0.853, train_s=0.930
Depth=24: train=0.972, test=0.918, train_s=0.984
Depth=32: train=0.990, test=0.929, train_s=0.998
Depth=40: train=0.992, test=0.929, train_s=1.000
Depth=None: train=0.992, test=0.930, train_s=1.000

Best depth: None
Best test accuracy: 0.9297649456131791
[[58997  3758     8     0   117    29   455]
 [ 4045 79732   407     2   608   359    79]
 [    7   189  9913   124    35   445     0]
 [    0     2    92   725     0    30     0]
 [   56   326    30     0  2324    17     1]
 [   12   146   476    56    16  4461     0]
 [  274    34     0     0     1     0  5827]]
              precision    recall  f1-score   support

           1      0.931     0.931     0.931     63364
           2      0.947     0.935     0.941     85232
           3      0.907     0.925     0.916     10713
           4      

In [78]:
depths = [3, 7, 12, 17, 24, 32, 40, None]
best_depth = None
best_test_acc = -np.inf
pca_best_tree = None

for d in depths:  
    tree = DecisionTreeClassifier(criterion = 'gini', max_depth= d, random_state=42)
    tree.fit(pca_X_smote, pca_y_smote)
    train_acc_s = tree.score(pca_X_smote, pca_y_smote)
    train_acc = tree.score(X_train_pca, y_train_pca)
    test_acc = tree.score(X_test_pca, y_test_pca)

    print(f"Depth={d}: train={train_acc:.3f}, test={test_acc:.3f}, train_s={train_acc_s:.3f}")
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_depth = d
        pca_best_tree = tree

print("\nBest depth:", best_depth)
print("Best test accuracy:", best_test_acc)
y_pred = pca_best_tree.predict(X_test_pca)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))
mcc = matthews_corrcoef(y_test_pca, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)


Depth=3: train=0.250, test=0.249, train_s=0.373
Depth=7: train=0.138, test=0.136, train_s=0.388
Depth=12: train=0.151, test=0.142, train_s=0.424
Depth=17: train=0.211, test=0.172, train_s=0.508
Depth=24: train=0.401, test=0.249, train_s=0.696
Depth=32: train=0.576, test=0.294, train_s=0.869
Depth=40: train=0.633, test=0.302, train_s=0.926
Depth=None: train=0.647, test=0.301, train_s=0.940

Best depth: 40
Best test accuracy: 0.30186838102344804
[[21255 13834  5170  1994  6781  3595 10735]
 [19295 24232  8630  3213 10845  6676 12341]
 [ 1187  1568  2815  1177  1289  1993   684]
 [   40    50   194   372    56   109    28]
 [  449   751   363   125   529   267   270]
 [  425   711  1328   505   561  1420   217]
 [ 1609  1095   372   145   684   264  1967]]
              precision    recall  f1-score   support

           1      0.480     0.335     0.395     63364
           2      0.574     0.284     0.380     85232
           3      0.149     0.263     0.190     10713
           4      0

In [79]:
depths = [3, 7, 12, 17, 24, 32, 40, None]
best_depth = None
best_test_acc = -np.inf
e_best_tree = None

for d in depths:  
    tree = DecisionTreeClassifier(criterion = 'gini', max_depth= d, random_state=42)
    tree.fit(eX_smote, ey_smote)
    train_acc_s = tree.score(eX_smote, ey_smote)
    train_acc = tree.score(eX_train, ey_train)
    test_acc = tree.score(eX_test, ey_test)

    print(f"Depth={d}: train={train_acc:.3f}, test={test_acc:.3f}, train_s={train_acc_s:.3f}")
    
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_depth = d
        e_best_tree = tree

print("\nBest depth:", best_depth)
print("Best test accuracy:", best_test_acc)
y_pred = e_best_tree.predict(eX_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))
mcc = matthews_corrcoef(ey_test, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)


Depth=3: train=0.318, test=0.315, train_s=0.595
Depth=7: train=0.585, test=0.583, train_s=0.718
Depth=12: train=0.723, test=0.712, train_s=0.825
Depth=17: train=0.846, test=0.805, train_s=0.903
Depth=24: train=0.954, test=0.859, train_s=0.974
Depth=32: train=0.981, test=0.865, train_s=0.997
Depth=40: train=0.983, test=0.865, train_s=1.000
Depth=None: train=0.983, test=0.866, train_s=1.000

Best depth: None
Best test accuracy: 0.8657693080389174
[[54679  7733    27     0   169    44   712]
 [ 7946 74954   582     3  1031   607   109]
 [   14   293  9035   220    62  1089     0]
 [    0     3   172   589     0    85     0]
 [   80   493    51     0  2102    27     1]
 [   16   252   934    78    19  3868     0]
 [  455    78     0     0     0     0  5603]]
              precision    recall  f1-score   support

           1      0.865     0.863     0.864     63364
           2      0.894     0.879     0.887     85232
           3      0.836     0.843     0.840     10713
           4      

In [80]:

# number of descriptive features
N = kX_train.shape[1]

# define k values
k_values = [
    int(2 * np.sqrt(N)),
    int(3 + np.sqrt(N)),
    int(np.sqrt(N)),
    max(1, int(np.sqrt(N) - 3))  # avoid k < 1
]

print("Candidate k values:", k_values)

best_k = None
best_test_acc = -np.inf
best_knn = None

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, metric="minkowski", weights="uniform")
    knn.fit(kX_train, y_train)

    train_acc = knn.score(kX_train, y_train)
    test_acc = knn.score(kX_test, y_test)

    print(f"k={k}: train={train_acc:.3f}, test={test_acc:.3f}")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_k = k
        best_knn = knn

print("\nBest k:", best_k)
print("Best test accuracy:", best_test_acc)

# Evaluate the best model
y_pred = best_knn.predict(kX_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)


Candidate k values: [14, 10, 7, 4]
k=14: train=0.771, test=0.735
k=10: train=0.792, test=0.743
k=7: train=0.821, test=0.756
k=4: train=0.862, test=0.769

Best k: 4
Best test accuracy: 0.7691702781046408
[[53575  8988    28     0   104    15   654]
 [16080 67214  1045     2   462   348    81]
 [   47  2071  7422   158    24   991     0]
 [    0     9   587   147     0   106     0]
 [  193  1781    52     0   720     8     0]
 [   34  1015  2514    44    16  1544     0]
 [ 2599   158     0     0     0     0  3379]]
              precision    recall  f1-score   support

           1      0.739     0.846     0.788     63364
           2      0.827     0.789     0.808     85232
           3      0.637     0.693     0.664     10713
           4      0.419     0.173     0.245       849
           5      0.543     0.261     0.353      2754
           6      0.513     0.299     0.378      5167
           7      0.821     0.551     0.659      6136

    accuracy                          0.769    

In [ ]:

# number of descriptive features
N = keX_smote.shape[1]

# define k values
k_values = [
    int(2 * np.sqrt(N)),
    int(3 + np.sqrt(N)),
    int(np.sqrt(N)),
    max(1, int(np.sqrt(N) - 3))  # avoid k < 1
]

print("Candidate k values:", k_values)

best_k = None
best_test_acc = -np.inf
best_knn = None

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, metric="minkowski", weights="uniform")
    knn.fit(keX_smote, key_smote)

    train_acc = knn.score(keX_smote, key_smote)
    test_acc = knn.score(kX_test, y_test)

    print(f"k={k}: train={train_acc:.3f}, test={test_acc:.3f}")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_k = k
        best_knn = knn

print("\nBest k:", best_k)
print("Best test accuracy:", best_test_acc)

# Evaluate the best model
y_pred = best_knn.predict(kX_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)


Candidate k values: [14, 10, 7, 4]


In [ ]:

# number of descriptive features
N = kX_smote.shape[1]

# define k values
k_values = [
    int(2 * np.sqrt(N)),
    int(3 + np.sqrt(N)),
    int(np.sqrt(N)),
    max(1, int(np.sqrt(N) - 3))  # avoid k < 1
]

print("Candidate k values:", k_values)

best_k = None
best_test_acc = -np.inf
best_knn = None

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, metric="minkowski", weights="uniform")
    knn.fit(kX_smote, ky_smote)

    train_acc = knn.score(kX_smote, ky_smote)
    test_acc = knn.score(kX_test, y_test)

    print(f"k={k}: train={train_acc:.3f}, test={test_acc:.3f}")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_k = k
        best_knn = knn

print("\nBest k:", best_k)
print("Best test accuracy:", best_test_acc)

# Evaluate the best model
y_pred = best_knn.predict(kX_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)


In [ ]:

# number of descriptive features
N = pca_kXsmote.shape[1]

# define k values
k_values = [
    int(2 * np.sqrt(N)),
    int(3 + np.sqrt(N)),
    int(np.sqrt(N)),
    max(1, int(np.sqrt(N) - 3))  # avoid k < 1
]

print("Candidate k values:", k_values)

best_k = None
best_test_acc = -np.inf
best_knn = None

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, metric="minkowski", weights="uniform")
    knn.fit(pca_kXsmote, pca_kysmote)

    train_acc = knn.score(pca_kXsmote, pca_kysmote)
    test_acc = knn.score(kX_test, y_test)

    print(f"k={k}: train={train_acc:.3f}, test={test_acc:.3f}")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_k = k
        best_knn = knn

print("\nBest k:", best_k)
print("Best test accuracy:", best_test_acc)

# Evaluate the best model
y_pred = best_knn.predict(kX_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient (MCC):", mcc)
